# Imports

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
btns = base.btns_gpio
from pynq.lib.arduino import Arduino_IO
import time
import multiprocessing
#import os
import socket
import threading

# Rotary encoder Microblaze C code

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin
void write_gpio(unsigned int pin, unsigned int val){
    gpio pin_out = gpio_open(pin); //declare a gpio object called pin_out. Use the open function to open it as a GPIO pin.
    gpio_set_direction(pin_out, GPIO_OUT); //Set the direction of the newly-defined pin_out gpio object to be an output.
    gpio_write(pin_out, val); //write the requested value by the function call to the pin.
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin) {
    gpio pin_in = gpio_open(pin); //declare a gpio object called pin_in. Use the open function to open it as a GPIO pin.
    gpio_set_direction(pin_in, GPIO_IN); //Set the direction of the newly-defined pin_in gpio object to be an input.
    return gpio_read(pin_in); //read the value currently on this GPIO pin
}

In [ ]:
#color codes:
CODE_RED = 0
CODE_BLU = 1
CODE_GRN = 2
CODE_TUR = 3
CODE_PUR = 4

#Define pins:
D0 = Arduino_IO(base.ARDUINO, 0, 'out')
D1 = Arduino_IO(base.ARDUINO, 1, 'out')
D2 = Arduino_IO(base.ARDUINO, 2, 'out')
D3 = Arduino_IO(base.ARDUINO, 3, 'out')
D4 = Arduino_IO(base.ARDUINO, 4, 'out')
D5 = Arduino_IO(base.ARDUINO, 5, 'out')
D6 = Arduino_IO(base.ARDUINO, 6, 'out')
D7 = Arduino_IO(base.ARDUINO, 7, 'out')
RS = Arduino_IO(base.ARDUINO, 8, 'out')
RW = Arduino_IO(base.ARDUINO, 9, 'out')
E = Arduino_IO(base.ARDUINO, 10, 'out')

#Constants:
OUTPUT_DIR = 1
INPUT_DIR = 0

READ = 1
WRITE = 0

TRUE = 1
FALSE = 0

INCREMENT = 1
DECREMENT = 0

NO_SHIFT = 0
SHIFT = 1

EIGHT_BIT_BUS_MODE = 1
FOUR_BIT_BUS_MODE = 0

TWO_LINE_MODE = 1
ONE_LINE_MODE = 0

FIVE_BY_EIGHT = 0
FIVE_BY_ELEVEN = 1

DISPLAY_ON = 1
DISPLAY_OFF = 0

CURSOR_ON = 1
CURSOR_OFF = 0

BLINK_ON = 1
BLINK_OFF = 0

NO_INITIAL_VALUE = 0

SLEEP_TIME_US = 10 #largest time in table 7.1 is 0.5us, so just use 10us for everything.

TEMP_TITLE_ADDR = 0
TEMP_ADDR = 6
TEMP_C_ADDR = 14
BRIGHTNESS_ADDR = 64
BRIGHTNESS_PERCENT_ADDR = 72
PERCENT_ADDR = 75
COLOR_ADDR = 77

#1602A Library Methods:

"""
EToggle() timing diagram:
//(1) set E=0, RS=RS, RW=0, DB0-7 set
//(2) wait 10us
//(3) set E=1
//(4) wait 10us
//(5) set E=0
//(6) wait 10us
"""
def EToggle():
    E.write(0)
    time.sleep(0.0001) #100us to be safe
    E.write(1)
    time.sleep(0.0001)
    E.write(0)
    time.sleep(0.0001)
    

"""
Writes '0x20' to DDRAM and sets DDRAM address to 0x00 from AC.
"""
def clearDisplay():
    RS.write(0)
    RW.write(WRITE)
    D7.write(0)
    D6.write(0)
    D5.write(0)
    D4.write(0)
    D3.write(0)
    D2.write(0)
    D1.write(0)
    D0.write(1)
    EToggle()
    
def returnHome():
    RS.write(0)
    RW.write(WRITE)
    D7.write(0)
    D6.write(0)
    D5.write(0)
    D4.write(0)
    D3.write(0)
    D2.write(0)
    D1.write(1)
    D0.write(1) #don't care
    EToggle()
    
def entryModeSet(incrementDecrement, shift):
    RS.write(0)
    RW.write(WRITE)
    E.write(0)
    D7.write(0)
    D6.write(0)
    D5.write(0)
    D4.write(0)
    D3.write(0)
    D2.write(0)
    D1.write(incrementDecrement)
    D0.write(shift)
    EToggle()

def displayControl(displayOn, cursorOn, blinkOn):
    RS.write(0)
    RW.write(WRITE)
    E.write(0)
    D7.write(0)
    D6.write(0)
    D5.write(0)
    D4.write(0)
    D3.write(1)
    D2.write(displayOn)
    D1.write(cursorOn)
    D0.write(blinkOn)
    EToggle()

def functionSet(busMode, numLines, charType):
    RS.write(0)
    RW.write(WRITE)
    E.write(0)
    D7.write(0)
    D6.write(0)
    D5.write(1)
    D4.write(busMode)
    D3.write(numLines)
    D2.write(charType)
    #D1.write(0) #don't care
    #D0.write(0) #don't care
    EToggle()
    
"""
Valid addresses are 0-39 for Line 1, 64 to 103 for Line 2
"""
def writeChar(character, address):
    #(1) Write DDRAM address:
    address = address & 0x000000FF;
    RS.write(0)
    RW.write(WRITE)
    E.write(0)
    D7.write(1)
    D6.write((address & 0b1000000)>>6)
    D5.write((address & 0b0100000)>>5)
    D4.write((address & 0b0010000)>>4)
    D3.write((address & 0b0001000)>>3)
    D2.write((address & 0b0000100)>>2)
    D1.write((address & 0b0000010)>>1)
    D0.write((address & 0b0000001)>>0)
    EToggle()
    
    #(2) Write character:
    character = ord(character) #convert single-element string into integer ASCII code
    RS.write(1)
    RW.write(WRITE)
    E.write(0)
    D7.write((character & 0b10000000)>>7)
    D6.write((character & 0b01000000)>>6)
    D5.write((character & 0b00100000)>>5)
    D4.write((character & 0b00010000)>>4)
    D3.write((character & 0b00001000)>>3)
    D2.write((character & 0b00000100)>>2)
    D1.write((character & 0b00000010)>>1)
    D0.write((character & 0b00000001)>>0)
    EToggle()
    
def writeText(textString, startAddress):
    ctr = 0
    for c in textString:
        writeChar(c, startAddress + ctr)
        ctr = ctr + 1

#LCD initialization:
displayControl(DISPLAY_ON, CURSOR_OFF, BLINK_OFF)
functionSet(EIGHT_BIT_BUS_MODE, TWO_LINE_MODE, FIVE_BY_EIGHT)
entryModeSet(INCREMENT, NO_SHIFT)
clearDisplay()
returnHome()
writeText("Temp: ", TEMP_TITLE_ADDR)
writeText("C", TEMP_C_ADDR)
writeText("Bright: ", BRIGHTNESS_ADDR)
writeText("%", PERCENT_ADDR)

#shm_a = shared_memory.SharedMemory(create=True, size=2)
#buffer = shm_a.buf
#buffer[0] = 
brightness = 5 #start brightness register at mid-level, or value 5 (scale 0-10)
#buffer[1] = 
color = CODE_TUR #default is turquoise

"""
def read_rotary_encoder(brightness):
    #global brightness
    pin_enc_A = 0 #CLK
    pin_enc_B = 1 #DT
    current_state = [read_gpio(pin_enc_A), read_gpio(pin_enc_B)]
    last_state = current_state
    #print("function started") #debug statement
    #while True:
    current_state = [read_gpio(pin_enc_A), read_gpio(pin_enc_B)]

    if last_state == [1, 1]:
        if current_state == [0, 1]: #CW rotation
            if brightness != 10: #if at 10, don't increment further
                brightness = brightness + 1
                print(brightness)
        elif current_state == [1, 0]: #CCW rotation
            if brightness != 0: #if at 0, don't decrement further
                brightness = brightness - 1
                print(brightness)
    elif last_state == [0, 0]:
        if current_state == [1, 0]: #CW rotation
            if brightness != 10: #if at 10, don't increment further
                brightness = brightness + 1
                print(brightness)
        elif current_state == [0, 1]: #CCW rotation
            if brightness != 0: #if at 0, don't decrement further
                brightness = brightness - 1
                print(brightness)
    last_state = current_state
    time.sleep(0.01) #sample rate is 100Hz
    return brightness
"""
def get_brightness():
    global brightness
    while True:
        if btns.read() == 2: #sensitive to BTN1
            if brightness != 10:
                brightness = brightness + 1
                print("Brightness: {}".format(brightness))
            time.sleep(0.2) #for button debouncing 
        elif btns.read() == 4: #sensitive to BTN2
            if brightness != 0:
                brightness = brightness - 1
                print("Brightness: {}".format(brightness))
            time.sleep(0.2) #for button debouncing 
        
def get_btn0():
    global color
    while True:
        if btns.read() == 1: #sensitive to BTN0
            if color == CODE_PUR:
                color = CODE_RED
            else:
                color = color + 1
            print("Color: {}".format(color))
            time.sleep(0.2) #for button debouncing
        
def server_side():
    global brightness, color
    #original_sigint = signal.getsignal(signal.SIGINT)
    #signal.signal(signal.SIGINT, exit)
    sock_l = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock_l.bind(('0.0.0.0', 1251))
    sock_l.listen()
    print('Waiting for connection')
    conn, addr = sock_l.accept()
    print('Connected')
    with conn:
        while True:
            #receive temperature data:
            temp_data = conn.recv(1024)
            writeText(temp_data.decode("utf-8"), TEMP_ADDR) #write to LCD
            
            #check for brightness from rotary encoder:
            pwm = brightness * 10
            conn.send(str(pwm).encode())
            writeText(str(pwm), BRIGHTNESS_PERCENT_ADDR) #write to LCD
            
            #check color:
            conn.send(str(color).encode())
            if color == 0: #red
                writeText("RED", COLOR_ADDR) #write to LCD, RED/GRN/BLU/PRP/TUR
            elif color == 1: #blue
                writeText("BLU", COLOR_ADDR)
            elif color == 2: #green
                writeText("GRN", COLOR_ADDR)
            elif color == 3: #turquoise
                writeText("TUR", COLOR_ADDR)
            elif color == 4: #purple
                writeText("PUR", COLOR_ADDR)

"""
# Launch rotary encoder process on CPU1
proc_rotenc = multiprocessing.Process(target=read_rotary_encoder) 
#os.system("taskset -p -c {} {}".format(1, proc_rotenc.pid)) # taskset is an os command to pin the process to a specific CPU
proc_rotenc.start() # start the process

# Launch btn0 process on CPU0
proc_btn0_color = multiprocessing.Process(target=get_btn0) 
#os.system("taskset -p -c {} {}".format(0, proc_btn0_color.pid)) # taskset is an os command to pin the process to a specific CPU
proc_btn0_color.start() # start the process

# Launch server process on CPU0
proc_server = multiprocessing.Process(target=server_side) 
#os.system("taskset -p -c {} {}".format(0, proc_server.pid)) # taskset is an os command to pin the process to a specific CPU
proc_server.start() # start the process

proc_rotenc.join() #"wait for this [thread/process] to complete"
print('rotary encoder process with name, {}, is finished'.format(proc_rotenc.name))
proc_btn0_color.join()
print('rotary encoder process with name, {}, is finished'.format(proc_rotenc.name))
proc_server.join()
print('rotary encoder process with name, {}, is finished'.format(proc_rotenc.name))
"""


# Launch rotary encoder process on CPU1
thread_rotenc = threading.Thread(target=get_brightness) 
thread_rotenc.start() # start the thread

# Launch btn0 process on CPU0
thread_btn0_color = threading.Thread(target=get_btn0) 
thread_btn0_color.start() # start the thread

# Launch server process on CPU0
thread_server = threading.Thread(target=server_side) 
thread_server.start() # start the thread

#thread_rotenc.join() #"wait for this [thread/process] to complete"
#print('rotary encoder thread with name, {}, is finished'.format(thread_rotenc.name))
thread_btn0_color.join()
print('rotary encoder thread with name, {}, is finished'.format(thread_btn0_color.name))
thread_server.join()
print('rotary encoder thread with name, {}, is finished'.format(thread_server.name))


Waiting for connection
Connected
Color: 4
Color: 0
Brightness: 6
Brightness: 7
Brightness: 6
Brightness: 5
